# Content based recommendation systems

### Project by : Ratnamala Bharath Kishore


Abstarct: 
Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of using Python and the Pandas library.


### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Content-Based Filtering</a></li>
    </ol>
</div>
<br>


<a id="ref1"></a>

# Acquiring the Data


Downloaded the dataset from Kaggle with the help of link given below:
https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset?select=IMDb+ratings.csv


Now you're ready to start working with the data!


<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [90]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:


In [99]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('IMDB movies.csv')
movies_df.dropna(inplace=True)
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('IMDb ratings.csv')
ratings_df.dropna(inplace=True)
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df=movies_df.drop(['title','date_published','duration','country','language','director','writer','production_company','actors','description','avg_vote','votes','budget','usa_gross_income','worlwide_gross_income','metascore','reviews_from_users','reviews_from_critics'],axis=1)


movies_df.head()
movies_df.count()


C:\Users\bhara\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


imdb_title_id     6616
original_title    6616
year              6616
genre             6616
dtype: int64

Let's also remove the year from the **title** column by using pandas' replace function and store in a new **year** column.


In [100]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
#movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
#movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['original_title'] = movies_df.original_title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['original_title'] = movies_df['original_title'].apply(lambda x: x.strip())
movies_df.head()

,imdb_title_id,original_title,year,genre
506,tt0017136,Metropolis,1927,"Drama, Sci-Fi"
1048,tt0021749,City Lights,1931,"Comedy, Drama, Romance"
2454,tt0027977,Modern Times,1936,"Comedy, Drama, Family"
2795,tt0029453,Pépé le Moko,1937,"Crime, Drama, Romance"
2827,tt0029583,Snow White and the Seven Dwarfs,1937,"Animation, Family, Fantasy"


With that, let's also split the values in the **Genres** column into a **list of Genres** to simplify future use. This can be achieved by applying Python's split string function on the correct column.


In [101]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genre']=movies_df['genre'].map(str)
movies_df['genre'] = movies_df.genre.str.split('|')
movies_df.head()

,imdb_title_id,original_title,year,genre
506,tt0017136,Metropolis,1927,"[Drama, Sci-Fi]"
1048,tt0021749,City Lights,1931,"[Comedy, Drama, Romance]"
2454,tt0027977,Modern Times,1936,"[Comedy, Drama, Family]"
2795,tt0029453,Pépé le Moko,1937,"[Crime, Drama, Romance]"
2827,tt0029583,Snow White and the Seven Dwarfs,1937,"[Animation, Family, Fantasy]"


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.


In [102]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genre']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,imdb_title_id,original_title,year,genre,"Drama, Sci-Fi","Comedy, Drama, Romance","Comedy, Drama, Family","Crime, Drama, Romance","Animation, Family, Fantasy","Drama, History, Romance",...,"Comedy, Music, War","Crime, Horror, Romance","Animation, Biography, Crime",Adventure,"Horror, Music, Thriller","Biography, Drama, Western","Animation, Drama, Horror","Action, History","Comedy, Fantasy, Music","Drama, Horror, Music"
506,tt0017136,Metropolis,1927,"[Drama, Sci-Fi]",1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048,tt0021749,City Lights,1931,"[Comedy, Drama, Romance]",0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2454,tt0027977,Modern Times,1936,"[Comedy, Drama, Family]",0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2795,tt0029453,Pépé le Moko,1937,"[Crime, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2827,tt0029583,Snow White and the Seven Dwarfs,1937,"[Animation, Family, Fantasy]",0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.


In [103]:
ratings_df.head()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0
24,tt0003740,7.1,3073,6.5,7.0,285,301,591,727,443,...,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0
47,tt0004972,6.3,22213,6.4,7.0,3661,1741,3314,3963,2876,...,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0
55,tt0005680,5.9,130,5.9,6.0,4,3,8,20,52,...,6.5,4.0,6.2,5.0,5.7,50.0,5.9,26.0,5.8,87.0


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.


<a id="ref3"></a>

# Content-Based recommendation system


Now, let's take a look at how to implement **Content-Based** or **Item-Item recommendation systems**. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the **userInput**. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [104]:
userInput = [
            {'original_title':'Breakfast Club, The', 'rating':5},
            {'original_title':'Toy Story', 'rating':3.5},
            {'original_title':'Jumanji', 'rating':2},
            {'original_title':"Pulp Fiction", 'rating':5},
            {'original_title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,original_title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user

With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [105]:
#Filtering out the movies by title
inputId = movies_df[movies_df['original_title'].isin(inputMovies['original_title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genre', 1).drop('year', 1)
#Final input dataframe
inputMovies

,imdb_title_id,original_title,rating
0,tt0110912,Pulp Fiction,5.0
1,tt0113497,Jumanji,2.0
2,tt0114709,Toy Story,3.5


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [106]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['imdb_title_id'].isin(inputMovies['imdb_title_id'].tolist())]
userMovies

,imdb_title_id,original_title,year,genre,"Drama, Sci-Fi","Comedy, Drama, Romance","Comedy, Drama, Family","Crime, Drama, Romance","Animation, Family, Fantasy","Drama, History, Romance",...,"Comedy, Music, War","Crime, Horror, Romance","Animation, Biography, Crime",Adventure,"Horror, Music, Thriller","Biography, Drama, Western","Animation, Drama, Horror","Action, History","Comedy, Fantasy, Music","Drama, Horror, Music"
28381,tt0110912,Pulp Fiction,1994,"[Crime, Drama]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29034,tt0113497,Jumanji,1995,"[Adventure, Comedy, Family]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29371,tt0114709,Toy Story,1995,"[Animation, Adventure, Comedy]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [107]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('imdb_title_id', 1).drop('original_title', 1).drop('genre', 1).drop('year', 1)
userGenreTable

,"Drama, Sci-Fi","Comedy, Drama, Romance","Comedy, Drama, Family","Crime, Drama, Romance","Animation, Family, Fantasy","Drama, History, Romance","Comedy, Drama","Adventure, Family, Fantasy","Animation, Comedy, Family","Drama, Mystery",...,"Comedy, Music, War","Crime, Horror, Romance","Animation, Biography, Crime",Adventure,"Horror, Music, Thriller","Biography, Drama, Western","Animation, Drama, Horror","Action, History","Comedy, Fantasy, Music","Drama, Horror, Music"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.


In [108]:
userGenreTable=userGenreTable.values.astype(float)

In [109]:
inputMovies['rating']

0    5.0
1    2.0
2    3.5
Name: rating, dtype: float64

In [110]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       3.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 2. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [112]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['imdb_title_id'])
#And drop the unnecessary information
genreTable = genreTable.drop('imdb_title_id', 1).drop('original_title', 1).drop('genre', 1).drop('year', 1)
genreTable.head()

,"Drama, Sci-Fi","Comedy, Drama, Romance","Comedy, Drama, Family","Crime, Drama, Romance","Animation, Family, Fantasy","Drama, History, Romance","Comedy, Drama","Adventure, Family, Fantasy","Animation, Comedy, Family","Drama, Mystery",...,"Comedy, Music, War","Crime, Horror, Romance","Animation, Biography, Crime",Adventure,"Horror, Music, Thriller","Biography, Drama, Western","Animation, Drama, Horror","Action, History","Comedy, Fantasy, Music","Drama, Horror, Music"
imdb_title_id,,,,,,,,,,,,,,,,,,,,,
tt0017136,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0021749,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0027977,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0029453,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tt0029583,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
genreTable.shape

(6616, 393)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [114]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

imdb_title_id
tt0017136    0.0
tt0021749    0.0
tt0027977    0.0
tt0029453    0.0
tt0029583    0.0
dtype: float64

In [115]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

imdb_title_id
tt0276919    0.47619
tt0304328    0.47619
tt0097240    0.47619
tt0924129    0.47619
tt1489167    0.47619
dtype: float64

Now here's the recommendation table!


In [116]:
#The final recommendation table
movies_df.loc[movies_df['imdb_title_id'].isin(recommendationTable_df.head(20).keys())]

,imdb_title_id,original_title,year,genre
11303,tt0056592,To Kill a Mockingbird,1962,"[Crime, Drama]"
16556,tt0071562,The Godfather: Part II,1974,"[Crime, Drama]"
19083,tt0079668,The Onion Field,1979,"[Crime, Drama]"
21047,tt0086066,The Outsiders,1983,"[Crime, Drama]"
21112,tt0086250,Scarface,1983,"[Crime, Drama]"
22409,tt0090670,At Close Range,1986,"[Crime, Drama]"
24546,tt0097240,Drugstore Cowboy,1989,"[Crime, Drama]"
27150,tt0106469,Bound by Honor,1993,"[Crime, Drama]"
28829,tt0112818,Dead Man Walking,1995,"[Crime, Drama]"
39690,tt0276919,Dogville,2003,"[Crime, Drama]"


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages

-   Learns user's preferences
-   Highly personalized for the user

##### Disadvantages

-   Doesn't take into account what others think of the item, so low quality item recommendations might happen
-   Extracting data is not always intuitive
-   Determining what characteristics of the item the user dislikes or likes is not always obvious
